* [Link To Code Page](https://github.com/BU-IE-582/fall19-omerburak/blob/master/IE582_HW4.ipynb)
* [To Download Python Code As Notebook](https://bu-ie-582.github.io/fall19-omerburak/IE582_HW4.ipynb)

In [1]:
import numpy as np
import pandas as pd

In [465]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import poisson
from scipy.stats import norm
from scipy.stats import skellam

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
%matplotlib inline

In [718]:
matches = pd.read_csv("matches.csv")

In [719]:
stats = pd.read_csv("stats.csv")

In [720]:
bets = pd.read_csv("bets.csv")

In [721]:
matches["match_total_score"]=matches["match_hometeam_score"]+matches["match_awayteam_score"]

In [722]:
# created scores data including team_id and team_score time ordered to find historical average scores

temp1 = matches[["match_awayteam_id", "match_awayteam_score", "match_total_score", "match_id", "epoch"]]
temp1 = temp1.rename(columns={"match_awayteam_id": "team_id", "match_awayteam_score": "team_score"})

temp2 = matches[["match_hometeam_id", "match_hometeam_score", "match_total_score", "match_id", "epoch"]]
temp2 = temp2.rename(columns={"match_hometeam_id": "team_id", "match_hometeam_score": "team_score"})

scores = pd.concat([temp1, temp2])
scores = scores.sort_values(by=["epoch"])

scores = scores.set_index([pd.Index(range(len(scores)))])

In [723]:
scores_temp = {}
for i, team_id in enumerate(scores["team_id"].unique()):
    scores_temp[i] = scores[scores["team_id"]==team_id][["team_score","match_total_score"]].rolling(5,min_periods=1).mean().shift(periods=1)
    scores_temp[i] = scores_temp[i].rename(columns={"team_score": "avg5_score", "match_total_score": "avg5_total_score"})

In [724]:
scores_avg=pd.concat([scores_temp[i] for i in range(len(scores_temp))]).sort_index()
scores = pd.concat([scores,scores_avg],axis=1)

In [725]:
scores.head()

,team_id,team_score,match_total_score,match_id,epoch,avg5_score,avg5_total_score
0,7109,1.0,2.0,41196,1505559600,NaN,NaN
1,7097,1.0,2.0,41196,1505559600,NaN,NaN
2,2614,1.0,1.0,13331,1505561400,NaN,NaN
3,2619,0.0,1.0,13331,1505561400,NaN,NaN
4,3224,2.0,3.0,17683,1505568600,NaN,NaN


In [726]:
#combined matches data with average scores created

matches=matches.merge(scores[["team_id","match_id","avg5_score","avg5_total_score"]], left_on=["match_awayteam_id","match_id"], right_on=["team_id","match_id"], how='left')
matches=matches.drop(columns=["team_id"])
matches=matches.rename(columns={"avg5_score": "away_avg5_score", "avg5_total_score": "away_avg5_total_score"})

matches=matches.merge(scores[["team_id","match_id","avg5_score","avg5_total_score"]], left_on=["match_hometeam_id","match_id"], right_on=["team_id","match_id"], how='left')
matches=matches.drop(columns=["team_id"])
matches=matches.rename(columns={"avg5_score": "home_avg5_score", "avg5_total_score": "home_avg5_total_score"})

#class number of having total goal over 2.5 or not
matches["many_goaled"]=(matches["match_total_score"]>2.5).astype(int)

In [727]:
stats=stats.drop(columns=["home_CornerKicks","home_Fouls","home_GoalkeeperSaves","home_Offsides","home_YellowCards","away_CornerKicks",
                    "away_Fouls","away_GoalkeeperSaves","away_Offsides","away_YellowCards","home_BlockedShots","away_BlockedShots",
                    "home_FreeKicks","away_FreeKicks","home_Throw-in","away_Throw-in","home_RedCards","away_RedCards","home_Tackles",
                    "home_TotalPasses","away_Tackles","away_TotalPasses","home_CompletedPasses","away_CompletedPasses","home_GoalKicks",
                    "away_GoalKicks","home_DistanceCovered(metres)","away_DistanceCovered(metres)","home_PassSuccess%","away_PassSuccess%"])

In [728]:
#add team and time info to the stat dataframe to get historical average of stats

stats=stats.merge(matches[['match_awayteam_id', 'match_hometeam_id', 'match_id', 'epoch']], on=["match_id"], how='left')

stats = stats.sort_values(by=["epoch"])
stats = stats.set_index([pd.Index(range(len(stats)))])

stats["home_BallPossession"] = stats["home_BallPossession"].str.replace("%", '').astype('float')
stats["away_BallPossession"] = stats["away_BallPossession"].str.replace("%", '').astype('float')

In [729]:
stats_temp = {}
for i, team_id in enumerate(stats["match_hometeam_id"].unique()):
    stats_temp[i] = stats[stats["match_hometeam_id"]==team_id][['home_BallPossession', 'home_GoalAttempts',
       'home_ShotsoffGoal', 'home_ShotsonGoal', 'away_BallPossession',
       'away_GoalAttempts', 'away_ShotsoffGoal', 'away_ShotsonGoal',
       'home_Attacks', 'home_DangerousAttacks', 'away_Attacks',
       'away_DangerousAttacks']].rolling(5,min_periods=1).mean().shift(periods=1)

In [730]:
stats_avg=pd.concat([stats_temp[i] for i in range(len(stats_temp))]).sort_index().rename(columns=dict([[i,"avg_"+i] for i in stats_avg.columns]))

In [731]:
stats=stats.drop(columns=['home_BallPossession', 'home_GoalAttempts',
       'home_ShotsoffGoal', 'home_ShotsonGoal', 'away_BallPossession',
       'away_GoalAttempts', 'away_ShotsoffGoal', 'away_ShotsonGoal',
       'home_Attacks', 'home_DangerousAttacks', 'away_Attacks',
       'away_DangerousAttacks'])

In [732]:
stats = pd.concat([stats,stats_avg],axis=1)

In [733]:
stats.head()

,match_id,match_awayteam_id,match_hometeam_id,epoch,avg_home_BallPossession,avg_home_GoalAttempts,avg_home_ShotsoffGoal,avg_home_ShotsonGoal,avg_away_BallPossession,avg_away_GoalAttempts,avg_away_ShotsoffGoal,avg_away_ShotsonGoal,avg_home_Attacks,avg_home_DangerousAttacks,avg_away_Attacks,avg_away_DangerousAttacks
0,41196,7109,7097,1505559600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13331,2614,2619,1505561400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17684,3235,3223,1505568600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17683,3224,3238,1505568600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17682,3237,3225,1505568600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [734]:
#get the bets of over 2.5 and under 2.5

bets = bets[bets["variable"].isin(["o+2.5","u+2.5"])]
bets = bets[["match_id","variable","value"]].groupby(["match_id","variable"]).mean()
bets = bets.unstack(level=-1)
bets.columns=['o+2.5', 'u+2.5']
bets.reset_index(level=0, inplace=True)

In [738]:
bets.head()

,match_id,o+2.5,u+2.5
0,145899,1.364500,2.845000
1,145900,1.425745,2.610213
2,149762,1.626600,2.173000
3,150851,1.768182,1.984545
4,150855,1.692857,2.104286


In [739]:
premier_league=matches[matches["league_id"]==148]
premier_league=premier_league[premier_league["match_status"]=="Finished"]
premier_league=premier_league[["match_awayteam_id","match_hometeam_id","match_id","epoch","match_total_score","many_goaled","away_avg5_score","away_avg5_total_score","home_avg5_score","home_avg5_total_score"]]

premier_league=premier_league.merge(stats, on=["match_hometeam_id","match_awayteam_id","match_id","epoch"], how='left')

premier_league=premier_league.merge(bets, on=["match_id"], how='left')

In [740]:
premier_league.head()

,match_awayteam_id,match_hometeam_id,match_id,epoch,match_total_score,many_goaled,away_avg5_score,away_avg5_total_score,home_avg5_score,home_avg5_total_score,...,avg_away_BallPossession,avg_away_GoalAttempts,avg_away_ShotsoffGoal,avg_away_ShotsonGoal,avg_home_Attacks,avg_home_DangerousAttacks,avg_away_Attacks,avg_away_DangerousAttacks,o+2.5,u+2.5
0,2614,2619,13331,1505561400,1.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2626,2623,13329,1505570400,6.0,1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2629,2621,13327,1505570400,2.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2641,2654,13456,1505570400,1.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2617,2616,13324,1505651400,0.0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [760]:
X=premier_league.drop(columns=['match_awayteam_id', 'match_hometeam_id', 'match_id', 'epoch', 'match_total_score', 'many_goaled'])
X=X.interpolate().fillna(X.interpolate().mean())
y=premier_league["match_total_score"]

In [808]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split

In [761]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [846]:
#PRA
#chosen alpha after 5fold cross validation
lasso_reg = LassoCV(cv=5, random_state=30).fit(X_train,y_train)
print("For lambda:",lasso_reg.alpha_,"scores:",lasso_reg.score(X_train, y_train),lasso_reg.score(X_test, y_test))

For lambda: 0.013795213077581121 scores: 0.06557461783110885 0.007848755217265468


In [833]:
#DT
min_obs_leaf = range(1,10)
for min_samples_leaf in min_obs_leaf:
    dt = DecisionTreeClassifier(min_samples_leaf=min_samples_leaf)
    dt.fit(X_train, y_train)
    print(dt.score(X_train, y_train),",",dt.score(X_test, y_test),"for","min_samples_leaf of",min_samples_leaf)

0.9935205183585313 , 0.1935483870967742 for min_samples_leaf of 1
0.7796976241900648 , 0.2064516129032258 for min_samples_leaf of 2
0.6997840172786177 , 0.1935483870967742 for min_samples_leaf of 3
0.6393088552915767 , 0.2064516129032258 for min_samples_leaf of 4
0.5982721382289417 , 0.23225806451612904 for min_samples_leaf of 5
0.5593952483801296 , 0.2064516129032258 for min_samples_leaf of 6
0.5485961123110151 , 0.1870967741935484 for min_samples_leaf of 7
0.5226781857451404 , 0.18064516129032257 for min_samples_leaf of 8
0.49892008639308855 , 0.1870967741935484 for min_samples_leaf of 9


In [845]:
#RF
features = range(1,X.shape[1])
for feat in features:
    rf = RandomForestRegressor(n_estimators=500,min_samples_leaf=5,max_features=feat,random_state=32)
    rf.fit(X_train, y_train)
    print("Selected features:",feat,rf.score(X_train, y_train),",",rf.score(X_test, y_test))

Selected features: 1 0.22972331990927186 , 0.013475029443762332
Selected features: 2 0.35967375541794355 , 0.00526381119313113
Selected features: 3 0.4102618680682263 , 0.0009959407392989839
Selected features: 4 0.4365705206652584 , 0.005928144093675036
Selected features: 5 0.45671520730676907 , -0.0026154835777452146
Selected features: 6 0.4705142267931666 , -0.0017062000325851479
Selected features: 7 0.47794384597983264 , 0.0008018153961983909
Selected features: 8 0.488905420270488 , -0.004067546390131627
Selected features: 9 0.49942308476234676 , -0.00016256756323418387
Selected features: 10 0.50373655019218 , 0.0024453821260316255
Selected features: 11 0.5100802585562207 , -0.011685343371965029
Selected features: 12 0.5157930005263807 , -0.006606637909736035
Selected features: 13 0.5177557704760782 , 0.0013726835133992532
Selected features: 14 0.5249283916630775 , -0.001258433419375038
Selected features: 15 0.5283914728434338 , -0.005383817878374053
Selected features: 16 0.53306126

In [841]:
#SGB
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]
trees = [10, 100, 1000]
depth = range(1,5)
for learning_rate in lr_list:
    for tree in trees:
        for max_depth in depth:
            gb_clf = GradientBoostingClassifier(n_estimators=tree, learning_rate=learning_rate, subsample=0.7, max_depth=max_depth, random_state=20)
            gb_clf.fit(X_train, y_train)
            print("For learning rate",learning_rate,tree,"trees",max_depth,"max_depth",":",gb_clf.score(X_train, y_train),",",gb_clf.score(X_test, y_test))

For learning rate 0.05 10 trees 1 max_depth : 0.3434125269978402 , 0.1935483870967742
For learning rate 0.05 10 trees 2 max_depth : 0.4168466522678186 , 0.2
For learning rate 0.05 10 trees 3 max_depth : 0.531317494600432 , 0.1870967741935484
For learning rate 0.05 10 trees 4 max_depth : 0.6673866090712743 , 0.1870967741935484
For learning rate 0.05 100 trees 1 max_depth : 0.4773218142548596 , 0.17419354838709677
For learning rate 0.05 100 trees 2 max_depth : 0.7580993520518359 , 0.17419354838709677
For learning rate 0.05 100 trees 3 max_depth : 0.9438444924406048 , 0.1870967741935484
For learning rate 0.05 100 trees 4 max_depth : 0.9870410367170627 , 0.17419354838709677
For learning rate 0.05 1000 trees 1 max_depth : 0.8272138228941684 , 0.14838709677419354
For learning rate 0.05 1000 trees 2 max_depth : 0.9935205183585313 , 0.14838709677419354
For learning rate 0.05 1000 trees 3 max_depth : 0.9935205183585313 , 0.17419354838709677
For learning rate 0.05 1000 trees 4 max_depth : 0.9935

## Regression Problem
### Generally sorting order by scores on training data:
##### SGB > DT > RF > PRA
### max training scores:
#### SGB:0.994, DT:0.994, RF:0.536, PRA:0.066
### max test scores:
#### SGB:0.232, DT:0.232 , RF:0.013, PRA:0.008
### When max training scores are chosen, test scores:
#### SGB:0.232, DT:0.194 , RF:-0.010, PRA:0.008
### CrossValidation error and test error is different on RF especially
### All the models are probably underfitted since test scores so low according to training scores with even max test score. Some training results may be too high which also indicates overfitting on training data for that parameters.

### The parameters selected according to cross-validation on training data:
#### SGB: learning rate 0.25, number of trees 1000, max_depth 3
#### DT: the minimal number of observations per tree leaf 1
#### RF: number of features 17
#### PRA: lambda 0.0138


In [848]:
y=premier_league["many_goaled"]

In [849]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [850]:
#PRA
#chosen alpha after 5fold cross validation
lasso_reg = LassoCV(cv=5, random_state=30).fit(X_train,y_train)
print("For lambda:",lasso_reg.alpha_,"scores:",lasso_reg.score(X_train, y_train),lasso_reg.score(X_test, y_test))

For lambda: 0.12340837432080033 scores: 0.004161689578749739 0.004744409523896875


In [851]:
#DT
min_obs_leaf = range(1,10)
for min_samples_leaf in min_obs_leaf:
    dt = DecisionTreeClassifier(min_samples_leaf=min_samples_leaf)
    dt.fit(X_train, y_train)
    print(dt.score(X_train, y_train),",",dt.score(X_test, y_test),"for","min_samples_leaf of",min_samples_leaf)

0.9956803455723542 , 0.5096774193548387 for min_samples_leaf of 1
0.9330453563714903 , 0.49032258064516127 for min_samples_leaf of 2
0.9114470842332614 , 0.4838709677419355 for min_samples_leaf of 3
0.8833693304535637 , 0.5225806451612903 for min_samples_leaf of 4
0.8531317494600432 , 0.4838709677419355 for min_samples_leaf of 5
0.8315334773218143 , 0.535483870967742 for min_samples_leaf of 6
0.8077753779697624 , 0.5032258064516129 for min_samples_leaf of 7
0.7926565874730022 , 0.4967741935483871 for min_samples_leaf of 8
0.7818574514038877 , 0.5032258064516129 for min_samples_leaf of 9


In [852]:
#RF
features = range(1,X.shape[1])
for feat in features:
    rf = RandomForestRegressor(n_estimators=500,min_samples_leaf=5,max_features=feat,random_state=32)
    rf.fit(X_train, y_train)
    print("Selected features:",feat,rf.score(X_train, y_train),",",rf.score(X_test, y_test))

Selected features: 1 0.2285821701893237 , 0.0010627966488913065
Selected features: 2 0.3670268180998023 , -0.00533177384022987
Selected features: 3 0.42160487103303634 , -0.007552092076448469
Selected features: 4 0.4525171051797905 , -0.007715139910969393
Selected features: 5 0.4658230626227501 , -0.007080011420876665
Selected features: 6 0.4814397383975325 , -0.015569455302841151
Selected features: 7 0.49406058756891147 , -0.01206710857533011
Selected features: 8 0.5000857223830855 , -0.0041609454879267815
Selected features: 9 0.507172468109397 , -0.01721503661922208
Selected features: 10 0.5127163655223388 , -0.009845954066717288
Selected features: 11 0.5159931415916539 , -0.008449535054301904
Selected features: 12 0.5208136677986026 , -0.012707300532879273
Selected features: 13 0.5265194098398605 , -0.021132157997239664
Selected features: 14 0.528457111276402 , -0.015357485529958746
Selected features: 15 0.5300997312308934 , -0.020104499327621594
Selected features: 16 0.532966699809

In [853]:
#SGB
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]
trees = [10, 100, 1000]
depth = range(1,5)
for learning_rate in lr_list:
    for tree in trees:
        for max_depth in depth:
            gb_clf = GradientBoostingClassifier(n_estimators=tree, learning_rate=learning_rate, subsample=0.7, max_depth=max_depth, random_state=20)
            gb_clf.fit(X_train, y_train)
            print("For learning rate",learning_rate,tree,"trees",max_depth,"max_depth",":",gb_clf.score(X_train, y_train),",",gb_clf.score(X_test, y_test))

For learning rate 0.05 10 trees 1 max_depth : 0.572354211663067 , 0.5483870967741935
For learning rate 0.05 10 trees 2 max_depth : 0.5766738660907127 , 0.5483870967741935
For learning rate 0.05 10 trees 3 max_depth : 0.5896328293736501 , 0.5419354838709678
For learning rate 0.05 10 trees 4 max_depth : 0.6652267818574514 , 0.535483870967742
For learning rate 0.05 100 trees 1 max_depth : 0.6198704103671706 , 0.5419354838709678
For learning rate 0.05 100 trees 2 max_depth : 0.7732181425485961 , 0.5225806451612903
For learning rate 0.05 100 trees 3 max_depth : 0.8920086393088553 , 0.5225806451612903
For learning rate 0.05 100 trees 4 max_depth : 0.9654427645788337 , 0.5096774193548387
For learning rate 0.05 1000 trees 1 max_depth : 0.7926565874730022 , 0.5096774193548387
For learning rate 0.05 1000 trees 2 max_depth : 0.9892008639308856 , 0.5161290322580645
For learning rate 0.05 1000 trees 3 max_depth : 0.9956803455723542 , 0.4967741935483871
For learning rate 0.05 1000 trees 4 max_depth 

## Classification Problem
### Generally sorting order by scores on training data:
##### SGB > DT > RF > PRA
### max training scores:
#### SGB:0.996, DT:0.996, RF:0.536, PRA:0.004
### max test scores:
#### SGB:0.574, DT:0.535 , RF:0.001, PRA:0.004
### When max training scores are chosen, test scores:
#### SGB:0.555, DT:0.510 , RF:-0.016, PRA:0.004
### CrossValidation error and test error is different on RF especially
### RFA and PRA are underfitted since test scores so low according to training scores with even max test score. SGB and DT also can be underfitted but it cannot be said easily. Some training results may be too high which also indicates overfitting on training data for that parameters.

### The parameters selected according to cross-validation on training data:
#### SGB: learning rate 0.075, number of trees 1000, max_depth 3
#### DT: the minimal number of observations per tree leaf 1
#### RF: number of features 17
#### PRA: lambda 0.123
